In [ ]:
%load_ext autoreload
%autoreload 2

import sys
import os

os.chdir("../../../../")

import yaml
import anndata as ad
import matplotlib.pyplot as plt
import seaborn as sns
import mudata as md
import muon as mu
import numpy as np
import scanpy as sc
from matplotlib import gridspec
import scanpy.external as sce
from scipy import sparse
from matplotlib.colors import LinearSegmentedColormap

from utils.utils import *
from utils.plotting import *

sc.settings.verbosity = 0
sc.settings.set_figure_params(
    dpi=100,
    facecolor="white",
    # color_map="YlGnBu",
    frameon=False,
)
import matplotlib as mpl

sc.set_figure_params(dpi=100)
sns.set(style="white")

In [ ]:
data_path = "data/anca/ustekinumab/ANCA_4PK4PB_ustekinumab.h5mu"
anca_data = md.read_h5mu(data_path)

In [ ]:
anca_data

In [ ]:
relevant_obs = ["patient", "sample", "tissue", "case", "orig.ident", "cell_type"]
anca_data.obs = anca_data.obs[relevant_obs]

In [ ]:
mod = "RNA"
rna_data = ad.AnnData(
    anca_data.mod[mod].layers["counts"].copy(),
    # obs=anca_data.obs.copy(),
    var=anca_data.mod[mod].var[[]],
    layers={"counts": anca_data.mod[mod].layers["counts"].copy()},
    obsm={
        "X_harmony": anca_data.obsm["X_harmony"].copy(),
        "X_umap": anca_data.obsm["X_umap"].copy(),
    },
)

In [ ]:
mod = "CITE"
cite_data = ad.AnnData(
    anca_data.mod[mod].layers["counts"].copy(),
    # obs=anca_data.obs.copy(),
    var=anca_data.mod[mod].var[[]],
    layers={"counts": anca_data.mod[mod].layers["counts"].copy()},
    obsm={
        "X_harmony": anca_data.obsm["X_harmony"].copy(),
        "X_umap": anca_data.obsm["X_umap"].copy(),
    },
)

In [ ]:
rna_data.obs = anca_data.obs.copy()
cite_data.obs = anca_data.obs.copy()

## Futher processing

In [ ]:
print(len(cite_data.var_names))
var_names = [name for name in cite_data.var_names if not "Hash" in name]
cite_data = cite_data[:, var_names].copy()
sc.pp.filter_genes(cite_data, min_cells=10)
print(len(cite_data.var_names))

In [ ]:
mod_cite = cite_data

get_adata_stats(mod_cite)

In [ ]:
sc.pp.calculate_qc_metrics(mod_cite, inplace=True, percent_top=None)
sns.displot(mod_cite.obs.n_genes_by_counts)

In [ ]:
sns.displot(mod_cite.obs.total_counts)

In [ ]:
log_normalize(rna_data)
protein_clr(cite_data)

scale(rna_data)
scale(cite_data)

In [ ]:
rna_data.X = rna_data.layers["log1p"].copy()
cite_data.X = cite_data.layers["clr"].copy()

## Refine annotation

In [ ]:
rna_data.obs.cell_type.value_counts()

In [ ]:
mapping = {
    "CD4_EM/RM": "CD4+ Teff",
    "CD4_CM": "CD4+ Tcm",
    "CD4_Naive": "CD4+ naive",
    "CD8_EM/RM": "CD8+ Teff",
    "CD4_CM/EM": "CD4+ Tcm/Tem",
    "CD8_CTL": "CD8+ Tem",
}

## Save everything

In [ ]:
anca_data_clean = md.MuData(
    {
        "rna": rna_data,
        "cite": cite_data,
    }
)

# save data
anca_data_clean.write_h5mu("data/anca/ustekinumab/ANCA_4PK4PB_ustekinumab_clean.h5mu")